# Chạy từng lệnh này trong terminal nha a
conda create -n varipred python=3.9 -y

conda activate varipred

pip install -r requirements.txt

pip install torch==2.0.0 torchvision==0.15.1 torchaudio==2.0.1 --index-url https://download.pytorch.org/whl/cu118

pip install fair-esm

# CONFIG

In [277]:
import main
import pandas as pd
import json

class Args:
    df_path = r"D:\Bio_sequence_Research_AITALAB\benchmark\task3_variant_prediction\VariPred\example\dataset"
    pred = "target"
    output = "VariPred_output"
    train = False

args = Args()

# INFERENCE VARIPRED

In [278]:
import os
import config
import importlib

# Reload modules to pick up latest changes
import utils
import main as main_module
importlib.reload(utils)
importlib.reload(main_module)

target_df = pd.read_csv(f"{args.df_path}/{args.pred}.csv")

# Clean sequences: remove rows with invalid amino acids (X, *, U, O, B, J, Z)
invalid_chars = set('XUOBZJ*')
def has_invalid_chars(seq):
    if pd.isna(seq):
        return True
    return any(c in seq for c in invalid_chars)

# Filter out rows with invalid characters in wt_seq or mt_seq
initial_rows = len(target_df)
target_df = target_df[~(target_df['wt_seq'].apply(has_invalid_chars) | target_df['mt_seq'].apply(has_invalid_chars))]

# Also ensure wt_aa and mt_aa are single characters
target_df = target_df[
    (target_df['wt_aa'].apply(lambda x: isinstance(x, str) and len(x) == 1)) &
    (target_df['mt_aa'].apply(lambda x: isinstance(x, str) and len(x) == 1))
]

target_df = target_df.reset_index(drop=True)  # Reset index to make it continuous
filtered_rows = len(target_df)
print(f'Filtered out {initial_rows - filtered_rows} rows (invalid sequences or wrong aa format)')
print(f'Remaining valid sequences: {len(target_df)}')

# Check if embeddings exist, if not, generate them
if not os.path.exists(f'{config.esm_storage_path}/{args.pred}.pt'):
    if len(target_df) > 0:
        print(f'Generating embeddings for {args.pred}.csv ({len(target_df)} valid sequences)...')
        target_df['label'] = -1  # Placeholder label for inference
        main_module.get_embeds(target_df, dataset=args.pred)
    else:
        print(f'No valid sequences found in {args.pred}.csv')
else:
    print(f'Embeddings for {args.pred}.pt already exist')

# Run inference
if os.path.exists(f'{config.esm_storage_path}/{args.pred}.pt'):
    main_module.run_VariPred(target_ds=args.pred, output=args.output)
else:
    print('Embeddings not found. Cannot run inference.')

Filtered out 1735 rows (invalid sequences or wrong aa format)
Remaining valid sequences: 2827
Embeddings for target.pt already exist
X_target shape:  (2827, 2561)


100%|██████████| 89/89 [00:00<00:00, 1480.23it/s]



Your prediction results are saved in ../example/output_results/VariPred_output.txt


# EVALUATION METRICS

In [279]:
import importlib
from calculate_metrics import evaluate_performance
import calculate_metrics
importlib.reload(calculate_metrics)

res_file = r'D:\Bio_sequence_Research_AITALAB\benchmark\task3_variant_prediction\VariPred\example\output_results\VariPred_output.txt' 
ori_file = r'D:\Bio_sequence_Research_AITALAB\benchmark\task3_variant_prediction\VariPred\example\dataset\test.parquet'
    
metrics = calculate_metrics.evaluate_performance(res_file, ori_file)
print(json.dumps(metrics, indent=4))

with open('evaluation_metrics.json', 'w') as f:
    json.dump(metrics, f, indent=4)

{
    "acc": 0.8068623983020871,
    "auc": 0.7615755024364926,
    "balanced_acc": 0.663243923185606,
    "f1_macro": 0.6821036320139191,
    "mcc": 0.3809631831821203,
    "precision": 0.6014150943396226,
    "recall": 0.40348101265822783,
    "specificity": 0.9230068337129841
}
